# Run WhisperAI using Google Colab's GPU

Enable GPU - select Runtime -> Change runtime type in the Menu bar. Under Hardware Accelerator, select GPU.

## Pip install dependencies

In [1]:
!pip install pyngrok
!pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.5 MB/s eta 0:00:00


## Imports

In [2]:
from flask import Flask, request
from pyngrok import ngrok
import json
import whisper

## Global variables

In [3]:
NGROK_AUTH_TOKEN = 'YourToken'
TRANSLATE_FILENAME = 'translate.wav'
TRANSCRIBE_FILENAME = 'transcribe.wav'
WHISPER_MODEL = 'small'
port = 5000
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok.connect(port).public_url

'https://7c71-34-143-194-44.ngrok-free.app'

## Load WhisperAI model

In [4]:
model = whisper.load_model(WHISPER_MODEL)

100%|███████████████████████████████████████| 461M/461M [00:06<00:00, 77.9MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

## Run WhisperAI and host it using ngrok


In [ ]:
app = Flask(__name__)

@app.route('/', methods=['GET'])
def test():
    response = {'status':'OK','message':'Test'}
    return json.dumps(response)

@app.route('/asr', methods=['POST'])
def asr():
    task = request.args.get('task')
    language = request.args.get('language')
    audio_data = request.files['audio_file']

    if task == 'transcribe':
      audio_data.save(TRANSCRIBE_FILENAME)
      result = model.transcribe(TRANSCRIBE_FILENAME)
      return json.dumps(result)

    elif task == 'translate':
      audio_data.save(TRANSLATE_FILENAME)
      result = model.transcribe(TRANSLATE_FILENAME, language='ja', task='translate')
      return json.dumps(result)

    else:
      return 'Record not found', 400


if __name__ == '__main__':
   app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on /asr [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-5-cff0b777e3fa>", line 24, in asr
    result = model.transcribe(TRANSLATE_FILENAME, language='ja', task='translate')
  File "/u